# Regresión y codificación de atributos binarios
**Autor:** José A. Troyano &nbsp;&nbsp;&nbsp; **Revisor:** Beatriz Pontes   &nbsp;&nbsp;&nbsp; **Última modificación:** 17/01/2019

## Contenido

1. <a href="#regresor">Entrenamiento de un regresor</a> <br>
    1.1 <a href="#lineal">Regresión lineal</a> <br>
    1.2 <a href="#estimadores">Uso de estimadores:</a> _fit_, _predict_, _cross_\__val_\__predict_ y _cross_\__val_\__score_
2. <a href="#evaluacion">Evaluación: _MAE_, _MSE_ y _R2_</a> <br>
3. <a href="#val_ausentes">Tratamiento de valores ausentes</a> <br>
    3.1 <a href="#sust_media">Sustitución por la media</a> <br>
    3.2 <a href="#sust_regresor">Sustitución usando un regresor</a><br>
4. <a href="#cod_discretos">Codificación de atributos discretos</a><br>
    4.1 <a href="#tipos_atributos">Tipos de atributos</a> <br>
    4.2 <a href="#label_encoding">Codificación _label encoding_</a><br>
    4.3 <a href="#one_hot">Codificación _one hot_</a> <br>
    4.4 <a href="#mas_atributos">Usando más atributos</a> <br>
------------------------------------------------------

En este notebook usaremos un dataset para entrenar un regresor de Sklearn y veremos distintas formas de evaluar la calidad del regresor obtenido. Veremos, además, cómo usar un regresor para rellenar valores ausentes de un atributo numérico. Y, por último, veremos cómo codificar atributos discretos mediante valores numéricos. Esto es necesario porque muchos de los algoritmos de aprendizaje solo soportan atributos numéricos. En concreto, Sklearn solo puede manejar atributos numéricos.

Empezaremos por importar todos los elementos que usaremos a lo largo del notebook:

In [1]:
import numpy as np
import pandas as pd

# Regresión y evaluación
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_predict, cross_val_score

Para comenzar, usaremos el dataset _concrete_, disponible en el repositorio UCI. El dataset contiene 1030 registros correspondientes a medidas de resistencia de hormigón. Los atributos se corresponden con las proporciones de la mezcla distintas muestras de hormigón y la edad (en días) de la muestra. La variable numérica a predecir es la resistencia de cada muestra.

In [3]:
# EJERCICIO: leer el fichero 'concrete.csv' y crear el dataframe 'X' para los atributos, y la serie 'y' para la clase
DATOS = pd.read_csv('./data/Concrete.csv')
X = pd.DataFrame(data=DATOS.iloc[:,0:8]) # every other column
y = pd.Series(DATOS.iloc[:,-1]) # strength

## 1. Entrenamiento de un regresor <a name="regresor"> </a>

Entrenar un regresor es muy simple en Sklearn. Al igual que ocurre con un clasificador, basta con crear un objeto del estimador que queramos entrenar y ejecutar el método <code>fit</code>. En este notebook usaremos uno de los regresores más comunes: <code>LinearRegression</code>.

### 1.1 Regresión Lineal <a name="lineal"> </a>

La Regresión Linenal es un modelo matemático usado para aproximar la relación entre una variable dependiente $y$, y las variables independientes $x_i$. El modelo se expresa con la siguiente fórmula:

$$
y \approx \alpha+\beta_1x_1+\beta_2x_2...+\beta_nx_n
$$

Sklearn proporciona distinos métodos para realizar regresión lineal. El más simple de ellos es el de los _mínimos cuadrados_ que es el que implementa el estimador <code>LinearRegression</code>. La técnica de los mínimos cuadrados se utiliza para determinar los coeficientes de una función de regresión que minimicen la suma de los cuadrados de los errores. Para una función de regresión lineal, se trataría de minimizar esta expresión:

$$
S = \sum (y - f(X))^2 = \sum (y - \alpha+\beta_1x_1+\beta_2x_2...+\beta_nx_n)^2
$$


### 1.2 Uso de estimadores <a name="estimadores"> </a>

In [8]:
# EJERCICIO: crear un estimador de la clase LinearRegression y entrenarlo con el dataset <X,y>
lr_clf = LinearRegression()
lr_clf.fit(X,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

Una vez entrenado un regresor, podemos usarlo para predecir la clase de un conjunto de instancias con el método <code>predict</code>.

In [9]:
# EJERCICIO: predecir la salida de los primeros 10 valores de X con el regresor entrenado anteriormente
lr_clf.predict(X[0:10])

array([53.46346329, 53.73475651, 56.81258504, 67.66368153, 60.91205585,
       26.85991563, 68.42076149, 29.92792448, 19.7781474 , 31.44208441])

Podemos aplicar validación cruzada para evaluar, de la misma forma que hicimos con la clasificación. Por defecto la métrica de evaluación es <code>r2_score</code> aunque, como veremos en la siguiente sección, hay más métricas implementadas en Sklearn.

In [10]:
# EJERCICIO: predecir la salida de todas las instancias mediante validación cruzada y guardar las prediccciones en y_pred
y_pred = cross_val_predict(lr_clf,X,y, cv = 10)
y_pred

array([53.31616047, 53.75523611, 61.10426739, ..., 26.69607519,
       29.23867412, 31.7853727 ])

In [11]:
# EJERCICIO: calcular la métrica r2_score sobre todas las instancias mediante validación cruzada
from sklearn.metrics import r2_score
r2_score(y, y_pred)

0.5274322794587907

## 2. Evaluación <a name="evaluacion"> </a>

En las tareas de clasificación las métricas de evaluación se basan en el número de aciertos de las predicciones. En la regresión, sin embargo, no se puede hablar de aciertos ya que las predicciones son numéricas y es muy improbable predecir exactamente el valor correcto. Lo importante para evaluar un regresor es medir la diferencia entre el valor real y el valor predicho. 

En esta sección tres de las métricas más populares para evaluar la calidad de los regresores:
- MAE: _mean absolute error_
- MSE: _mean squared error_
- R2: coeficiente de determinación

Las fórmulas para cada una de las tres métricas son:

$$
MAE = \frac{\sum |\;y -f(X)\;|}{n}
$$

$$
MSE = \frac{\sum (y -f(X))^2}{n}
$$

$$
R2 = 1 - \frac{\sum (y -f(X))^2}{\sum (\bar{y} - y)}
$$

In [12]:
# EJERCICIO: dadas los siguientes vectores 'y_real' e 'y_pred' calcular las métricas MAE, MSE y R2
#    y_real = [1,   0.5, 1.5, -1]
#    y_pred = [1.5, 0,   1.5,  1]

y_real = [1,   0.5, 1.5, -1]
y_pred = [1.5, 0,   1.5,  1]
print("R2 score: ", r2_score(y_real,y_pred))

from sklearn.metrics import mean_absolute_error
print("MAE: " , mean_absolute_error(y_real,y_pred))

from sklearn.metrics import mean_squared_error
print("MSE: ", mean_squared_error(y_real,y_pred))

R2 score:  -0.2857142857142858
MAE:  0.75
MSE:  1.125


In [13]:
# EJERCICIO: calcular las métricas MAE, MSE y R2 mediante validación cruzada para el dataset 'concrete' y LinearRegression
# NOTA: los scores MAE y MSE son negativos para que los valores altos se correspondan con mejores resultados
y_pred_r2 = cross_val_score(lr_clf,X,y, scoring = 'r2', cv = 5)
y_pred_mae = cross_val_score(lr_clf,X,y, scoring = 'neg_mean_absolute_error', cv = 5)
y_pred_mse = cross_val_score(lr_clf,X,y, scoring = 'neg_mean_squared_error', cv = 5)

print("r2 scoring: ", y_pred_r2.mean())
print("mae scoring: ", y_pred_mae.mean())
print("mse scoring: ", y_pred_mse.mean())

r2 scoring:  0.46099404916628633
mae scoring:  -8.92539956910218
mse scoring:  -128.13775612964704


## 3. Tratamiento de valores ausentes <a name="val_ausentes"> </a>

En esta sección utilizaremos un regresor para estimar los valores ausentes de un atributo. En ocasiones esta técnica puede funcionar, pero no es una solución aplicable de forma general. Para que funcione realmente debe haber cierta redundancia entre los atributos del dataset, de esta forma podrá aprenderse cierta relación entre el atributo ausente y el resto de atributos. Usamos regresión porque el atributo con valores ausentes es numérico, si fuese un atributo discreto deberíamos usar un clasificador.

Haremos las pruebas con un dataset de calificaciones de estudiantes. Cada registro contendrá cinco informaciones (el curso en el que el estudiante inició sus estudios y cuatro calificaciones parciales) y la variable a estimar es la calificación final. En total hay 95 registros, y hay dos versiones del dataset:
- <code>class-grades.csv</code>: con los valores de todos los atributos.
- <code>incomplete-class-grades.csv</code>: el atributo <code>Midterm</code> solo está presente en 76 de las 95 instancias.

Evaluaremos tres versiones del dataset:
- Completo
- Incompleto en el que los valores ausentes se sustituyen por la media del atributo <code>Midterm</code>
- Incompleto en el que los valores ausentes se sustituyen mediante un regresor sobre el resto de atributo (no sobre la clase de salida)

Nos apoyaremos en la siguiente función para evaluar cada versión del dataset:

In [ ]:
# Función que, dado un dataset, calcula 'X' e 'y', y evalua la predicción de 'Final' mediante validación cruzada
def evalua(datos):
    y = datos['Final']
    X = datos.drop(['Final'], axis=1)
    clasificador = LinearRegression()
    scores = cross_val_score(clasificador, X, y, cv=10)
    print(scores.mean())
    
# Es mejor cuanto más se acerca a a 1 ya que el cross_val_score usa por defecto r2

In [ ]:
# EJERCICIO: leer el fichero 'class-grades.csv' y guardarlo en el dataset 'datos_completos' 
datos_completos = pd.read_csv('./class-grades.csv')
datos_completos

In [ ]:
# EJERCICIO: leer el fichero 'incomplete-class-grades.csv' y guardarlo en el dataset 'datos_incompletos' 
datos_incompletos = pd.read_csv('./incomplete-class-grades.csv')
datos_incompletos

In [ ]:
# EJERCICIO:evaluar el dataset con los datos completos
evalua(datos_completos)

In [ ]:
# EJERCICIO: ¿qué ocurre cuando se intenta evaluar el dataset con datos ausentes?
evalua(datos_incompletos)

### 3.1 Sustitución por la media <a name="sust_media"> </a>

In [ ]:
# EJERCICIO: realizar los siguientes pasos para evaluar la técnica de sustitución de valores ausentes mediante la media:
#    - Hacer una copia del dataset 'datos_incompletos' en 'rellenos_con_media'
#    - Calcular la media del atributo 'Midterm'
#    - Sustituir los valores ausentes por la media
#    - Evaluar el dataset resultante

rellenos_con_media = datos_incompletos.copy()
midter_mean = rellenos_con_media['Midterm'].mean()
#rellenos_con_media.isna().any() TRUE para Midterm
rellenos_con_media['Midterm'].fillna(midter_mean, inplace=True)
evalua(rellenos_con_media)

### 3.2 Sustitución usando un regresor <a name="sust_regresor"> </a>

In [ ]:
# EJERCICIO: realizar los siguientes pasos para entrenar un regresor que prediga 'Midterm' en función de otras columnas
#    - Crear la lista de posibles predictores: predictores_midterm = ['Prefix', Assignment', 'Tutorial', TakeHome']
#    - Crear 'X_train' e 'y_train' con los predictores anteriores y solo para las filas completas
#    - Entrenar un regresor lineal a partir de 'X_train' e 'y_train'

predictores_midterm = ['Prefix', 'Assignment', 'Tutorial', 'TakeHome']
solo_completos = datos_incompletos[~np.isnan(datos_incompletos['Midterm'])].copy()
X_train = solo_completos[predictores_midterm]
y_train = solo_completos['Midterm']

r1 = LinearRegression()
r1.fit(X_train,y_train)

La siguiente función calculará siempre un valor para el atributo <code>Midterm</code>. Si no está definido, se apoyará en el resto de atributos y en el modelo de regresión para estimar uno:

In [ ]:
def calcula_midterm(fila, modelo, columnas):
    if np.isnan(fila['Midterm']):
        atributos = fila[columnas]
        return modelo.predict([atributos])[0]
    else:
        return fila['Midterm']

In [ ]:
# EJERCICIO: realizar los siguientes pasos para evaluar la técnica de sustitución de valores ausentes mediante regresion:
#    - Hacer una copia del dataset 'datos_incompletos' en 'rellenos_con_regresion'
#    - Sustituir los valores ausentes con la función calcula_midterm
#    - Evaluar el dataset resultante
#    - ¿Qué combinación de 'predictores_midterm' funciona mejor?

rellenos_con_regresion = datos_incompletos.copy()
rellenos_con_regresion['Midterm'] = rellenos_con_regresion.apply(calcula_midterm,axis=1,args=(r1,predictores_midterm))
evalua(rellenos_con_regresion)

## 4. Codificación de atributos discretos <a name="cod_discretos"> </a>

En esta sección usaremos el dataset automobile, disponible en el repositorio UCI. El dataset original tiene algunos valores ausentes, pero nosotros trabajaremos con una versión en la que se han eliminado algunas filas y columnas para que no haya ningún valor ausente. En nuestra versión, el dataset contiene 197 filas con 22 atributos que describen características de coches y un atributo numérico price que será el que usaremos para entrenar un regresor.

In [ ]:
# Empezaremos por leer el dataset y crear 'X' e 'y'
datos = pd.read_csv('./automobile.csv')
y = datos['price']
X = datos.drop(['price'], axis=1)
print(X.info())

### 4.1. Tipos de atributos y _baseline_ con atributos numéricos <a name="tipos_atributos"> </a>

En esta sección separaremos el dataframe según el tipo de atributo y evaluaremos un regresor usando solo los atributos numéricos.


In [ ]:
# EJERCICIO: crear las matrices 'X_discretos' y 'X_numericos' con los atributos discretos y numéricos, respectivamente, de 'X'
X_discretos = X.select_dtypes(['object'])
X_numericos = X.select_dtypes(['number'])

In [ ]:
# EJERCICIO: definir una función 'evalua_dataset' que haga lo siguiente:
#    - Reciba como parámetros X e y
#    - Evalue mediante validación cruzada un RandomForestRegressor con n_estimators=200 y random_state=0
#    - Devuelva el resultado de la evaluación

def evalua_dataset(X,y):
    clasificador = RandomForestRegressor(n_estimators=200, random_state=0)
    scores = cross_val_score(clasificador, X, y, cv=10)
    return scores.mean()

In [ ]:
# TEST: de la función evalua_dataset
print(evalua_dataset(X_numericos, y))

In [ ]:
# EJERCICIO: mostrar la frecuencia de aparición de los valores de los atributos discretos
columnas = X_discretos.columns
for c in columnas:
    print("Columna: ", c ,"  ", np.unique(X_discretos[c],return_counts=True))

Según estos resultados nos encontramos con tres tipos de atributos discretos:
- **Binarios**: <code>['fuel-type', 'aspiration', 'engine-location']</code>
- **Categóricos**: <code>['make', 'body-style', 'drive-wheels', 'engine-type', 'fuel-system']</code>
- **Ordinales**: <code>['num-of-doors', 'num-of-cylinders']</code>

Los atributos ordinales se pueden codificar mediante un único atributo numérico, ya que la relación de orden se mantiene en la representación numérica. A este tipo de codificación se le denomina _label encoding_.

Los categóricos, sin embargo, no se pueden codificar con un número, ya que el algoritmo de aprendizaje asumiría una relación de orden que no existe. En este caso se debe utilizar una codificación en varias columnas, el denominado _one-hot encoding_.

Los binarios son realmente categóricos, pero podemos intentar codificarlos con un único atributo numérico que tome los valores $0$ y $1$.

### 4.2. Codificación _label encoding_ <a name="label_encoding"> </a>

La codificación _label encoding_ consiste en sustituir cada valor del atributo discreto por un valor numérico. Sklearn proporciona métodos para hacerlo, pero nosostros lo haremos directamente con el método map() de Pandas ya que se trata de un proceso bastante simple.

In [ ]:
# EJERCICIO: codificar mediante label encoding los atributos ordinales y los binarios
#    - Almacenar los nuevos atributos en una nueva matriz 'X_label'
#    - Elegir un valor numérco apropiado para cada valor discreto
#    - Se puede aplicar el método map() de las Series para realizar la correspondencia

fuel_labeled = X_discretos['fuel-type'].map({'diesel': 0, 'gas': 1})
aspiration_labeled = X_discretos['aspiration'].map({'std': 0, 'turbo': 1})
engine_location_labeled = X_discretos['engine-location'].map({'front': 0, 'rear': 1})
num_of_doors_labeled = X_discretos['num-of-doors'].map({'four': 4, 'two': 2})
num_of_cylinders_labeled = X_discretos['num-of-cylinders'].map({'eight':8, 'five':5, 'four':4, 'six':6, 'three':3, 'twelve':12, 'two':2})

matriz = np.stack([fuel_labeled,aspiration_labeled,engine_location_labeled,num_of_doors_labeled,num_of_cylinders_labeled],axis=1)
X_label= pd.DataFrame(matriz,columns=['fuel-type','aspiration','engine-location','num-of-doors','num-of-cylinders'])
X_label

In [ ]:
# TEST: del label_encoding de los atributos binarios y ordinales
for atributo in ['fuel-type', 'aspiration', 'engine-location', 'num-of-doors', 'num-of-cylinders']:
    print(list(X_discretos[atributo].head(10)))
    print(list(X_label[atributo].head(10)))
    print()

In [ ]:
# EJERCICIO: Evaluación de la contribución de cada atributo codificado con respecto al baseline de atributos numéricos


### 4.3. Codificación _one hot_ <a name="one_hot"> </a>

La codificación _one hot_ da lugar (para un atributo discreto) a tantas columnas como valores distintos tenga el atributo. Hay varias alternativas para realizar esta codificación (Pandas, Sklearn, Patsy, ...). Nosotros usaremos Pandas porque su sintaxis es muy sencilla.

In [ ]:
# EJERCICIO: codificar mediante one hot encoding los atributos categóricos
#    - Pandas proporciona un método para hacerlo
X_one_hot = pd.get_dummies(X_discretos,columns=['make', 'body-style', 'drive-wheels', 'engine-type', 'fuel-system'])
X_one_hot

In [ ]:
# TEST: de la codificación one hot
print(list(X_discretos['make'].head(10)))
print(list(X_one_hot['make_audi'].head(10)))
print(X_one_hot.info())

In [ ]:
# EJERCICIO: crear una función 'filtra_columnas' que seleccione solo las columnas correspondientes a un grupo de atributos
    ''' Construye un dataset solo con las columnas 'one hot' correspondientes al conjunto de atributos indicado
    
    Recibe:
       - datos_one_hot: dataset con columnas codificadas mediante one hot
       - atributos: nombres originales de los atributos a mantener
    '''

In [ ]:
# TEST: de la función 'filtra_columnas'
print(filtra_columnas(X_one_hot, ['make', 'body-style', 'fuel-system']).info())

In [ ]:
# EJERCICIO: Evaluación de la contribución de cada atributo codificado con respecto al baseline de atributos numéricos


### 4.4. Usando más atributos <a name="mas_atributos"> </a>

En esta última sección crearemos un dataset con los atributos numéricos, más la codificación de todos los atributos discretos. También haremos una prueba incluyendo un subconjunto de los atributos discretos.

In [ ]:
# EJERCICIO: crear un dataset 'X_eval' con los atributos numéricos, más todos los discretos codificados
#    - Partir de los siguientes data frames: 
#         X_numericos
#         X_label
#         X_one_hot
#    - Las listas de los nombres originales de los atributos codificados son:
#         cols_label =  ['fuel-type', 'aspiration', 'engine-location', 'num-of-doors', 'num-of-cylinders']
#         cols_one_hot = ['make', 'body-style', 'drive-wheels', 'engine-type', 'fuel-system']
#    - En el caso de los atributos one hot hay que usar la función 'filtra_columnas' para seleccionar los atributos
#    - Guardar dicho dataset en el fichero 'automobile-coded.csv'


In [ ]:
# EJERCICIO: evalua el regresor con el dataset de atributos numéricos y todos los discretos codificados


In [ ]:
# EJERCICIO: crear un dataset 'X_eval' con los atributos numéricos, más todos los discretos codificados
#         cols_label =  ['fuel-type', 'aspiration', 'engine-location', 'num-of-doors']
#         cols_one_hot = ['make', 'body-style', 'fuel-system']


In [ ]:
# EJERCICIO: evalua el regresor con el dataset de atributos numéricos y los mejores discretos codificados


Los resultados mejoran en ambos casos pero, curiosamente, con menos atributos se consiguen mejores resultados. Esto nos da pie a introducir la tarea que veremos en el siguiente notebook, la _selección de características_, que consiste en econtrar el subconjunto de atributos que mejor se comporta.